<a id="top"></a>
# Cloud-Filtered Custom Mosaics <img align="right" style="padding:10px" src="../images/odc_logo.png">

This notebook can be used to create custom Landsat cloud-filtered mosaics for any time period and location. The mosaics can be output as GeoTIFF products for analysis in external GIS tools. The following mosaics are possible: Median (midpoint of spectral data) and Max-NDVI (maximum vegetation respons). These mosaics are quite common and will be of value to a large number of users. 

Users should review the "Cloud Statistics" notebook for more information about the temporal and spatial sample. An understanding of the underlying data is important for creating a valid mosaic for further analyses. In many cases, cloud contamination will create poor mosaics. With a careful review of the cloud coverage over a given region and time period, it is possible to improve the mosaics and avoid false outputs. 

## <span id="import">Import Dependencies and Connect to the Data Cube [&#9652;](#top)</span>

In [2]:
# Add utils to the sys.path so that data_cube_utilities can be found. (Local folder hack.)
import sys
#sys.path.append('/mnt/userdata/notebooks/bkdev/utils')
sys.path.append('/mnt/userdata/notebooks/bkdev')

%matplotlib inline

# Supress Warning 
import warnings
warnings.filterwarnings('ignore')

# Load Data Cube Configuration
import datacube
import utils.data_cube_utilities.data_access_api as dc_api  

import xarray as xr  
import numpy as np
import matplotlib.pyplot as plt

api = dc_api.DataAccessApi(config = '/home/localuser/.datacube.conf')
dc = api.dc

## <span id="plat_prod">Choose Platforms and Products [&#9652;](#top)</span>

In [ ]:
# Select a Product and Platform
# Examples: ghana, kenya, tanzania, sierra_leone, senegal

# product = "ls7_ledaps_kenya"
# platform = "LANDSAT_7"

# product = "ls8_lasrc_kenya"
# platform = "LANDSAT_8"

# product = "ls7_ledaps_ghana"
# platform = "LANDSAT_7"

product = "ls8_lasrc_ghana"
platform = "LANDSAT_8"

## <span id="extents">Get the Extents of the Cube [&#9652;](#top)</span>

In [ ]:
# Print extents of the Data Cube so we know what data is available
extents = api.get_full_dataset_extent(platform = platform, product = product)

latitude_extents = (min(extents['latitude'].values),max(extents['latitude'].values))
longitude_extents = (min(extents['longitude'].values),max(extents['longitude'].values))
time_extents = (min(extents['time'].values),max(extents['time'].values))

print(latitude_extents)
print(longitude_extents)
print(time_extents)

## <span id="define_extents">Define the Extents of the Analysis [&#9652;](#top)</span>

In [ ]:
# Select an analysis region (Lat-Lon) within the extents listed above. 
# HINT: Keep your region small (<0.5 deg square) to avoid memory overload issues
# Select a time period (Min-Max) within the extents listed above (Year-Month-Day)

# Mombasa, Kenya
# latitude = (-4.1, -3.9) 
# longitude = (39.5, 39.7) 

# Mau Forest - Western Kenya
# latitude = (-0.13406, 0.21307)
# longitude = (35.28322, 35.56681)

# Tano-Offin Forest - Ghana
# latitude = (6.5814, 6.8978 ) 
# longitude = (-2.2955, -1.9395) 

# Mining Region near Obuasi, Ghana
latitude = (6.0985, 6.2675)
longitude = (-2.050, -1.8629)

# Time Period
time_extents = ('2019-01-01', '2019-03-31')

In [ ]:
# The code below renders a map that can be used to view the region.
from data_cube_utilities.dc_display_map import display_map
display_map(latitude,longitude)

## <span id="load_data">Load and Clean Data from the Data Cube [&#9652;](#top)</span>

In [ ]:
landsat_dataset = dc.load(latitude = latitude,
                          longitude = longitude,
                          platform = platform,
                          time = time_extents,
                          product = product,
                          measurements = ['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'pixel_qa']) 

In [ ]:
# Displays the first few values of each data array to check the content
# Latitude and Longitude numbers = number of pixels in each dimension
# Time = number of time slices in the dataset

landsat_dataset

In [ ]:
from utils.data_cube_utilities.clean_mask import landsat_qa_clean_mask
cloud_mask = landsat_qa_clean_mask(landsat_dataset, platform=platform)
cleaned_dataset = landsat_dataset.where(cloud_mask)

## <span id="mosaics">Create Mosaics and View the Results [&#9652;](#top)</span>

> **Median Mosaic**  
>  Masks clouds from imagery and use the median-valued cloud-free pixels in the time series.
>  More specifically, each band (e.g. red) of each pixel is assigned its median across time.
>  So this mosaic method generates values that are not in the dataset.
>
> **Max NDVI Mosaic**  
>  Masks clouds from imagery and use the Max NDVI across time for cloud-free pixels in the time series. 
>  The maximum NDVI will represent the highest amount of green vegetation 

In [ ]:
from utils.data_cube_utilities.dc_mosaic import create_median_mosaic, create_max_ndvi_mosaic
from utils.data_cube_utilities.dc_rgb import rgb

In [ ]:
median_composite = create_median_mosaic(cleaned_dataset, cloud_mask)
max_ndvi_composite = create_max_ndvi_mosaic(cleaned_dataset, cloud_mask.values)

In [ ]:
# Show two mosaics ... Median and Maximum NDVI

# RGB image options
# Standard RGB = 321 = Red, Green, Blue
# False Color = 543 = SWIR1, NIR, Red
# False Color (Landsat Mosaic) = 742 = SWIR2, NIR, Green

fig, ax = plt.subplots(1, 2, figsize=(16, 8))
median_rgb = median_composite[['swir2', 'nir', 'green']].to_array()
maxndvi_rgb = max_ndvi_composite[['swir2', 'nir', 'green']].to_array()
median_rgb.plot.imshow(ax=ax[0], vmin=0, vmax=5000)
maxndvi_rgb.plot.imshow(ax=ax[1], vmin=0, vmax=5000)
ax[0].set_title('Median Mosaic'), ax[0].xaxis.set_visible(False), ax[0].yaxis.set_visible(False)
ax[1].set_title('Max NDVI Mosaic'), ax[1].xaxis.set_visible(False), ax[1].yaxis.set_visible(False)
plt.show()

## <span id="export">Create GeoTIFF Output Products [&#9652;](#top)</span>

In [ ]:
from utils.data_cube_utilities.import_export import export_slice_to_geotiff

In [ ]:
# Remove the comment tags (#) to export a GeoTIFF output product
# Change the name of the output file, or it will be overwritten for each run 

# export_slice_to_geotiff(median_composite, 'geotiffs/DEMO_median_composite.tif')
# export_slice_to_geotiff(max_ndvi_composite, 'geotiffs/DEMO_max_ndvi_composite.tif')

In [ ]:
# Remove the comment tag (#) below to view the contents of the output folder
# !ls -lah geotiffs/